# Preprocessing B4B data

In [ ]:
import pandas as pd
import numpy as np
import pylab as plt

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

%load_ext autoreload
%autoreload 2
%matplotlib widget

from preprocessor import Preprocessor

In [ ]:
%%time
df = pd.read_parquet('b4b_raw_properties.parquet', engine='pyarrow').sort_index()

In [ ]:
%matplotlib widget
df.temp_in_degC.plot.hist(bins=50, alpha=0.5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df

## Experiments with filtering data

In [ ]:
Preprocessor.filter_min_max(df, 'temp_in_degC', min=5, max=45, inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
df_filt = Preprocessor.filter_static_outliers(df, 'temp_in_degC', n_sigma=3, inplace=False, per_id=True)

In [ ]:
%matplotlib widget
df.temp_in_degC.plot.hist(bins=50, alpha=0.5)

In [ ]:
%matplotlib widget
df_filt.temp_in_degC.plot.hist(bins=50, alpha=0.5)

In [ ]:
df_filt.info()

In [ ]:
df_filt.describe()

In [ ]:
df_filt

## Experiments with plotting data

In [ ]:
# This cell can be used to plot one or more properties in one or more rooms for one or more sources, not fully working yet

for id in list(df_plot.index.unique(level='id')):
    for source in list(df_plot.index.unique(level='source')):
        if len(df_plot.loc[id,source]):
            df_plot.loc[id,source].plot(
                # subplots=
                # [
                #     # ('co2_ppm'),
                #      # ('occupancy_p'),
                #      # ('valve_frac_0', 'rel_humidity_0'),
                #      # ('window_open_bool', 'door_open_bool'),
                #      ('temp_in_degC')
                # ],
                style='.--',
                title=f'room: {id}, source: {source}'
               )

In [ ]:
%autoreload 2
for id in list(df.index.unique(level='id')):
    for prop in list(df.columns):

        Plot.temperature_and_power_one_home_plot(f'{prop} in {id}',
                                        df.loc[id],
                                        temp_plot_dict = {prop: 'r'}
                                       )
        
        for device_type in list(df.index.unique(level='device_type')):
            df_plot = df.loc[id,:,device_type].copy()
            df_plot = Preprocessor.filter_min_max(df_plot, prop, min=0, max=45)
            df_plot = Preprocessor.filter_static_outliers(df_plot, prop, n_sigma=3)
            Plot.temperature_and_power_one_home_plot(f'{device_type}:{prop} in {id}',
                                        df_plot,
                                        temp_plot_dict = {prop: 'r'}
                                       )

In [ ]:
#cell to experiment with other property filtering code

id = list(df.index.unique(level='id'))[0]
prop = list(df.columns)[0]
device_type = list(df.index.unique(level='device_type'))[0]
dfplot = df.loc[id,:,device_type]

#insert new filtering code here

Plot.temperature_and_power_one_home_plot(f'{device_type}:{prop} in {id}',
                            dfplot,
                            temp_plot_dict = {prop: 'r'}
                           )

In [ ]:
dfplot.plot.hist(bins=200)